In [2]:
import numpy as np
import pandas as pd
from collections import Counter, defaultdict
import json

In [33]:
master = pd.read_csv("./ufc-master.csv")

In [34]:
fighters = np.unique(np.concatenate([np.asarray(master.R_fighter), np.asarray(master.B_fighter)]))

In [36]:
men = np.unique(np.concatenate([np.asarray(master[master.gender == "MALE"].R_fighter), np.asarray(master[master.gender == "MALE"].B_fighter)]))

In [39]:
women = np.unique(np.concatenate([np.asarray(master[master.gender == "FEMALE"].R_fighter), np.asarray(master[master.gender == "FEMALE"].B_fighter)]))

In [50]:
fights = defaultdict(Counter().copy)

In [51]:
visited = set()
for i, fighter in enumerate(men):
    right_opps = np.asarray(master[master.R_fighter == fighter].B_fighter)
    left_opps = np.asarray (master[master.B_fighter == fighter].R_fighter)

    left_and_right = np.concatenate([left_opps,right_opps])
    for j, opponent in enumerate(left_and_right):
        if opponent not in visited:    
            # Symmetric 2d Array
            visited.add(fighter)
            fights[fighter][opponent] +=1
            fights[opponent][fighter] +=1
        

In [54]:
pre_json = dict()
pre_json["nodes"] = dict()
nodes = []
for fighter in men:
    weight = np.concatenate([master[master.B_fighter == fighter].weight_class, master[master.R_fighter == fighter].weight_class])[0]
    cur = dict()
    cur["id"] = fighter
    cur["group"] = weight
    nodes.append(cur)
pre_json["nodes"] = nodes

In [55]:
links = list()
visited = set()
for fighter in fights:
    visited.add(fighter)
    for opp in fights[fighter]:
        if opp not in visited and opp not in women:
            cur = dict()
            cur["source"] = fighter
            cur["target"] = opp
            cur["value"] = fights[fighter][opp]
            links.append(cur)
whole = dict()
whole["nodes"] = nodes
whole["links"] = links

In [56]:
with open('fights.json', 'w') as outfile:
    json.dump(whole, outfile)

### Heavyweights

In [57]:
hw_master = master[master.weight_class == "Heavyweight"]
hw_master
HWs = np.unique(np.concatenate([np.asarray(hw_master.R_fighter), np.asarray(hw_master.B_fighter)]))
hw_fights = defaultdict(Counter().copy)

In [58]:
visited = set()
for i, fighter in enumerate(HWs):
    right_opps = np.asarray(hw_master[hw_master.R_fighter == fighter].B_fighter)
    left_opps = np.asarray (hw_master[hw_master.B_fighter == fighter].R_fighter)

    left_and_right = np.concatenate([left_opps,right_opps])
    for j, opponent in enumerate(left_and_right):
        if opponent not in visited and opponent in HWs:    
            # Symmetric 2d Array
            visited.add(fighter)
            hw_fights[fighter][opponent] +=1
            hw_fights[opponent][fighter] +=1

In [59]:
pre_json = dict()
pre_json["nodes"] = dict()
nodes = []
for fighter in HWs:
    weight = "Heavyweight"
    cur = dict()
    cur["id"] = fighter
    cur["group"] = weight
    nodes.append(cur)
pre_json["nodes"] = nodes

In [60]:
links = list()
visited = set()
for fighter in hw_fights:
    for opp in hw_fights[fighter]:
        if opp not in visited and opp in HWs :
            visited.add(fighter)
            cur = dict()
            cur["source"] = fighter
            cur["target"] = opp
            cur["value"] = hw_fights[fighter][opp]
            links.append(cur)
whole = dict()
whole["nodes"] = nodes
whole["links"] = links
whole

{'nodes': [{'id': 'Aaron Rosa', 'group': 'Heavyweight'},
  {'id': 'Adam Milstead', 'group': 'Heavyweight'},
  {'id': 'Adam Wieczorek', 'group': 'Heavyweight'},
  {'id': 'Alan Baudot', 'group': 'Heavyweight'},
  {'id': 'Alekander Volkov', 'group': 'Heavyweight'},
  {'id': 'Aleksei Oleinik', 'group': 'Heavyweight'},
  {'id': 'Alexander Gustafsson', 'group': 'Heavyweight'},
  {'id': 'Alexander Volkov', 'group': 'Heavyweight'},
  {'id': 'Alexandr Romanov', 'group': 'Heavyweight'},
  {'id': 'Alistair Overeem', 'group': 'Heavyweight'},
  {'id': 'Allen Crowder', 'group': 'Heavyweight'},
  {'id': 'Andrei Arlovski', 'group': 'Heavyweight'},
  {'id': 'Anthony Hamilton', 'group': 'Heavyweight'},
  {'id': 'Antonio Carlos Junior', 'group': 'Heavyweight'},
  {'id': 'Antonio Silva', 'group': 'Heavyweight'},
  {'id': 'Arjan Bhullar', 'group': 'Heavyweight'},
  {'id': 'Augusto Sakai', 'group': 'Heavyweight'},
  {'id': 'Ben Rothwell', 'group': 'Heavyweight'},
  {'id': 'Ben Sosoli', 'group': 'Heavyweight

In [1]:
import json
with open('heavyweights.json', 'w') as outfile:
    json.dump(whole, outfile)

NameError: name 'whole' is not defined

#### Rankings

In [3]:
ranks = pd.read_csv('./rankings_history.csv')

In [4]:
ranks

,date,weightclass,fighter,rank
0,2013-02-04,Pound-for-Pound,Anderson Silva,1
1,2013-02-04,Pound-for-Pound,Jon Jones,2
2,2013-02-04,Pound-for-Pound,Georges St-Pierre,3
3,2013-02-04,Pound-for-Pound,José Aldo,4
4,2013-02-04,Pound-for-Pound,Benson Henderson,5
...,...,...,...,...
63281,2022-03-29,Women's Bantamweight,Pannie Kianzad,12
63282,2022-03-29,Women's Bantamweight,Karol Rosa,13
63283,2022-03-29,Women's Bantamweight,Julia Avila,14
63284,2022-03-29,Women's Bantamweight,Norma Dumont,15


In [8]:
hw_ranks = ranks[ranks.weightclass == "Heavyweight"]

In [9]:
hw_ranks

,date,weightclass,fighter,rank
87,2013-02-04,Heavyweight,Cain Velasquez,0
88,2013-02-04,Heavyweight,Junior Dos Santos,1
89,2013-02-04,Heavyweight,Fabricio Werdum,2
90,2013-02-04,Heavyweight,Daniel Cormier,3
91,2013-02-04,Heavyweight,Antonio Silva,4
...,...,...,...,...
63232,2022-03-29,Heavyweight,Sergei Pavlovich,11
63233,2022-03-29,Heavyweight,Shamil Abdurakhimov,12
63234,2022-03-29,Heavyweight,Augusto Sakai,13
63235,2022-03-29,Heavyweight,Walt Harris,14


In [11]:
hw_ranks.to_csv("./hw_ranks.csv")

### Get seperate HW JSON

In [3]:
with open("fights.json") as input_file:
    all = json.load(input_file)

In [16]:
non_heavy = []
hw = []
for fighter in all['nodes']:
    if fighter['group'] != "Heavyweight":
        non_heavy.append(fighter['id'])
    else:
        hw.append(fighter)

In [17]:
hw_fights = []
for fight in all['links']:
    if fight['source'] not in non_heavy and fight['target'] not in non_heavy:
        hw_fights.append(fight)


In [21]:
hw_pre_json = {}
hw_pre_json['nodes'] = hw
hw_pre_json['links'] = hw_fights

In [23]:
with open("./hw_fights.json", "w") as output_file:
    json.dump(hw_pre_json, output_file)